In [ ]:
import pandas as pd
import requests
import json

observe_spot = '90	93	95	98	99	100	101	102	104	105	106	108	112	114	115	119	121	127	129	130	131	133	135	136	137	138	140	143	146	152	155	156	159	162	165	168	169	170	172	174	177	184	185	188	189	192	201	202	203	211	212	216	217	221	226	232	235	236	238	239	243	244	245	247	248	251	252	253	254	255	257	258	259	260	261	262	263	264	266	268	271	272	273	276	277	278	279	281	283	284	285	288	289	294	295'
observe = observe_spot.split()
translate = [[] for i in range(13)]

url = ''

for k in observe:
  print(k, 'start')

  for j in range(9):
    print(k, j, 'start')
    params ={'serviceKey' : '', 'pageNo' : str(j + 1), 'numOfRows' : '999', 'dataType' : 'JSON', 'dataCd' : 'ASOS', 'dateCd' : 'DAY', 'startDt' : '20000101', 'endDt' : '20250430', 'stnIds' : k }

    response = requests.get(url, params=params)

    js = json.loads(response.content)

    head = js['response']['header']

    if head['resultMsg'] != 'NORMAL_SERVICE':
       continue

    pharsed_data = js['response']['body']['items']['item']

    for i in pharsed_data:
      translate[0].append(i['tm']) # date
      translate[1].append(i['avgPa']) # pressure
      translate[2].append(i['maxTa']) # maxtemp
      translate[3].append(i['avgTa']) # temparature
      translate[4].append(i['minTa']) # mintemp
      translate[5].append(i['avgTd']) # dewpoint
      translate[6].append(i['avgRhm']) # humidity
      translate[7].append(i['avgTca']) # cloud
      translate[8].append(i['sumSsHr']) # sunshine
      translate[9].append(i['maxWd']) # winddirection
      translate[10].append(i['maxWs']) # windspeed
      translate[11].append(i['sumRn']) # rainfall
      translate[12].append(k) # region
    print(k, j, 'finish')
  print(k, 'finish')


95
90 start
90 0 start


KeyboardInterrupt: 

In [ ]:
print(response.content)
print(js)

b'<OpenAPI_ServiceResponse>\n\t<cmmMsgHeader>\n\t\t<errMsg>SERVICE ERROR</errMsg>\n\t\t<returnAuthMsg>HTTP ROUTING ERROR</returnAuthMsg>\n\t\t<returnReasonCode>04</returnReasonCode>\n\t</cmmMsgHeader>\n</OpenAPI_ServiceResponse>'


In [ ]:
for i in range(12):
  for j in range(len(translate[i])):
    if translate[i][j] == '':
      translate[i][j] = 0.0

In [ ]:
d = dict()

d['date'] = translate[0]
d['pressure'] = list(map(float, translate[1]))
d['maxtemp'] = list(map(float, translate[2]))
d['temparature'] = list(map(float, translate[3]))
d['mintemp'] = list(map(float, translate[4]))
d['dewpoint'] = list(map(float, translate[5]))
d['humidity'] = list(map(float, translate[6]))

d['cloud'] = list(map(lambda x: float(x)*10, translate[7]))

d['sunshine'] = list(map(float, translate[8]))
d['winddirection'] = list(map(int, translate[9]))
d['windspeed'] = list(map(float, translate[10]))
d['rainfall'] = list(map(float, translate[11]))
d['region'] = translate[12]

In [ ]:
weather = pd.DataFrame(d)
weather.head()

In [ ]:
weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95709 entries, 0 to 95708
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   date           95709 non-null  object 
 1   pressure       95709 non-null  float64
 2   maxtemp        95709 non-null  float64
 3   temparature    95709 non-null  float64
 4   mintemp        95709 non-null  float64
 5   dewpoint       95709 non-null  float64
 6   humidity       95709 non-null  float64
 7   cloud          95709 non-null  float64
 8   sunshine       95709 non-null  float64
 9   winddirection  95709 non-null  int64  
 10  windspeed      95709 non-null  float64
 11  rainfall       95709 non-null  float64
dtypes: float64(10), int64(1), object(1)
memory usage: 8.8+ MB


In [ ]:
weather.to_csv('/content/sample_data/weather.csv', sep=',', na_rep='NaN')